In [1]:
from roma_dspy import RecursiveSolverModule, RecursiveSolver, Executor, Atomizer, Planner, Aggregator
import dspy 

executor_lm = dspy.LM("openrouter/google/gemini-2.5-flash", cache=False)
atomizer_lm = dspy.LM("openrouter/google/gemini-2.5-flash", cache=False)
planner_lm = dspy.LM("openrouter/google/gemini-2.5-flash", cache=False)
aggregator_lm = dspy.LM("openrouter/google/gemini-2.5-flash", cache=False)

# Initialize modules
atomizer = Atomizer(lm=atomizer_lm)
planner = Planner(lm=planner_lm)
executor = Executor(lm=executor_lm)
aggregator = Aggregator(lm=aggregator_lm)

# Create solver
solver = RecursiveSolver(
    atomizer,
    planner,
    executor,
    aggregator,
    max_depth=1
)

dspy_module = RecursiveSolverModule(solver=solver)

In [ ]:
task_goal = "Write me a blog post about the benefits of using DSPy."

root = await dspy_module.aforward(task_goal)

In [ ]:
from roma_dspy.visualizer import TreeVisualizer, ContextFlowVisualizer, StatisticsVisualizer, HierarchicalVisualizer# Get the DAG for detailed visualization

# Show the full execution report
tree = TreeVisualizer(use_colors=True, show_timing=True)
print(tree.visualize(root.completed_task, dag=dspy_module._solver.last_dag))